# Ch `09`: Concept `03`

## Convolution Neural Network

Load data from CIFAR-10.

In [5]:
import numpy as np
import matplotlib.pyplot as plt
import cifar_tools
import tensorflow as tf

learning_rate = 0.001


#Load the dataset
names, data, labels = cifar_tools.read_data('https://github.com/netzo92/cs291k---MP1/tree/master/cifar-10-batches-py')

#     cifar_tools.read_data('C:/Users/howar/ch09_cnn/cifar-10-batches-py')

NameError: name 'cifar_tools' is not defined

Define the placeholders and variables for the CNN model:

In [ ]:
#Define the input and output placeholders
x = tf.placeholder(tf.float32, [None, 24 * 24])
y = tf.placeholder(tf.float32, [None, len(names)])

#Apply 64 convolutions of window-size 5x5
with tf.name_scope('layer1'):
    with tf.name_scope('layer1_weight'):
        W1 = tf.Variable(tf.random_normal([5, 5, 1, 64]))
    with tf.name_scope('layer1_bias'):    
        b1 = tf.Variable(tf.random_normal([64]))

#Then apply 64 more convolutions of window-size 5x5
with tf.name_scope('layer2'):
    with tf.name_scope('layer2_weight'):
        W2 = tf.Variable(tf.random_normal([5, 5, 64, 64]))
    with tf.name_scope('layer2_bias'):
        b2 = tf.Variable(tf.random_normal([64]))

#Then we introduce a fully-connected layer
with tf.name_scope('fc_layer'):
    with tf.name_scope('fc_weight'):
        W3 = tf.Variable(tf.random_normal([6*6*64, 1024]))
    with tf.name_scope('fc_bias'):
        b3 = tf.Variable(tf.random_normal([1024]))

#Lastly, define the variables for a fully-connected linear layer
with tf.name_scope('Output_layer'):
    with tf.name_scope('Output_weight'):
        W_out = tf.Variable(tf.random_normal([1024, len(names)]))
    with tf.name_scope('Output_bias'):
        b_out = tf.Variable(tf.random_normal([len(names)]))

Define helper functions for the convolution and maxpool layers:

In [ ]:
def conv_layer(x, W, b):
    conv = tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')
    conv_with_b = tf.nn.bias_add(conv, b)
    conv_out = tf.nn.relu(conv_with_b)
    return conv_out


def maxpool_layer(conv, k=2):
    return tf.nn.max_pool(conv, ksize=[1, k, k, 1], strides=[1, k, k, 1], padding='SAME')

The CNN model is defined all within the following method:

In [ ]:
def model():
    x_reshaped = tf.reshape(x, shape=[-1, 24, 24, 1])

    #Construct the first layer of convolution and maxpooling
    with tf.name_scope('layer1'):
        with tf.name_scope('conv_layer1'):
            conv_out1 = conv_layer(x_reshaped, W1, b1)
        with tf.name_scope('maxpool_layer1'):
            maxpool_out1 = maxpool_layer(conv_out1)
        with tf.name_scope('norm_layer1'):
            norm1 = tf.nn.lrn(maxpool_out1, 4, bias=1.0, alpha=0.001 / 9.0, beta=0.75)
    
    #Construct the second layer
    with tf.name_scope('layer2'):
        with tf.name_scope('conv_layer2'):
            conv_out2 = conv_layer(norm1, W2, b2)
        with tf.name_scope('norm_layer2'):
            norm2 = tf.nn.lrn(conv_out2, 4, bias=1.0, alpha=0.001 / 9.0, beta=0.75)
        with tf.name_scope('maxpool_layer2'):
            maxpool_out2 = maxpool_layer(norm2)

    #Lastly, construct the concluding fully connected layers
    with tf.name_scope('fully_connected'):
        with tf.name_scope('maxpool_layer2_reshaped'):
            maxpool_reshaped = tf.reshape(maxpool_out2, [-1, W3.get_shape().as_list()[0]])
        with tf.name_scope('fc_out_before_relu'):
            local = tf.add(tf.matmul(maxpool_reshaped, W3), b3)
        with tf.name_scope('fc_out'):
            local_out = tf.nn.relu(local)

    with tf.name_scope('total_out'):    
        out = tf.add(tf.matmul(local_out, W_out), b_out)
    return out

Here's the cost function to train the classifier.

In [ ]:
model_op = model()

cost = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits=model_op, labels=y)
)
train_op = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

correct_pred = tf.equal(tf.argmax(model_op, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

Let's train the classifier on our data:

In [ ]:
with tf.Session() as sess:
    # 初始化 Graph
    sess.run(tf.global_variables_initializer())
    
    
    # 將視覺化輸出
    writer = tf.summary.FileWriter("TensorBoard/", graph = sess.graph)
    onehot_labels = tf.one_hot(labels, len(names), on_value=1., off_value=0., axis=-1)
    onehot_vals = sess.run(onehot_labels)
    batch_size = len(data) // 200
    print('batch size', batch_size)
    
    #Loop through 1000 epochs
    for j in range(0, 1000):
        avg_accuracy_val = 0.
        batch_count = 0.
        
        #Train the network in batches
        for i in range(0, len(data), batch_size):
            batch_data = data[i:i+batch_size, :]
            batch_onehot_vals = onehot_vals[i:i+batch_size, :]
            _, accuracy_val = sess.run([train_op, accuracy], feed_dict={x: batch_data, y: batch_onehot_vals})
            avg_accuracy_val += accuracy_val
            batch_count += 1.
        avg_accuracy_val /= batch_count
        print('Epoch {}. Avg accuracy {}'.format(j, avg_accuracy_val))
